In [97]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import sys
sys.path.append('../')

In [99]:
import os
from src.pcai_datamodule_ca import PatchColorAdaptDataModule
from src.pcai_litmodule import LitModuleClasAdversarial

In [100]:
dataset_dir = '../data/tma_dataset/'
image_base_dir = os.path.join(dataset_dir, "images")
mask_base_dir = os.path.join(dataset_dir, "masks/masks/")
metadata_path = os.path.join(dataset_dir, 'metadata/metadata.csv')

## 1) Create histograms

In [101]:
hist_datamodule = PatchColorAdaptDataModule(
    metadata_path=metadata_path,
    image_base_dir=image_base_dir,
    mask_base_dir=mask_base_dir,  
    label_cols_clas=['event_duration', 'event_indicator'],
    label_col_domain=['domain'],
    patch_size=14,
)

histograms = hist_datamodule.create_slide_histograms()

Output()

Training with variable bag-size requires batch-size of 1!


/home/fabian/UKE_projects/PCAI/pcai/notebook/../src/pcai_datamodule_ca.py:190: TqdmExperimentalWarning: rich is experimental/alpha
  for patch_bag, _, _, meta_dict in (pbar := rich_tqdm(histograms_dataloader)):


In [102]:
import pickle

with open("histograms.pkl", "wb") as f:
    pickle.dump(histograms, f)

## 2) Setup prediction

In [103]:
datamodule = PatchColorAdaptDataModule(
    metadata_path=metadata_path,
    image_base_dir=image_base_dir,
    mask_base_dir=mask_base_dir,
    label_cols_clas=['event_duration', 'event_indicator'],
    label_col_domain=['domain'],
    patch_size=14,
    n_patches=5,
    batch_size=2,
    histograms_path="histograms.pkl",
    cp_preds_path="predictions_cp.pkl",
)

datamodule.setup("fit")

Computing 8 kmeans clusters...
Computing nearest cluster for all 9 samples...


In [104]:
sample = next(iter(datamodule.train_dataloader()))
len(sample)

4

In [105]:
litmodule = LitModuleClasAdversarial()

In [106]:
from pytorch_lightning import Trainer

trainer = Trainer(accelerator="cpu")

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [107]:
predictions = trainer.predict(model=litmodule, datamodule=datamodule)

Computing 8 kmeans clusters...
Computing nearest cluster for all 9 samples...


/home/fabian/anaconda3/envs/pcai-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

Matching histogram of sample 0_test with cluster 0
Matching histogram of sample 1_test with cluster 1
Matching histogram of sample 2_test with cluster 2


In [108]:
from src.utils import output_collate, transform_collated_output

predictions_transformed = transform_collated_output(output_collate(predictions))
predictions_transformed

{'scalar':    x|clas|pred_cls  x|clas|prob_cls_0  x|clas|prob_cls_1  x|adv|pred_cls  \
 0                1           0.497683           0.502317               2   
 1                1           0.497683           0.502317               2   
 2                1           0.497683           0.502317               2   
 3                1           0.497683           0.502317               2   
 4                1           0.497683           0.502317               2   
 5                1           0.497683           0.502317               2   
 6                1           0.497683           0.502317               2   
 7                1           0.497683           0.502317               2   
 8                1           0.497683           0.502317               2   
 
    x|adv|prob_cls_0  x|adv|prob_cls_1  x|adv|prob_cls_2  y|clas  y|adv  \
 0          0.325133          0.337257           0.33761       0      0   
 1          0.325133          0.337257           0.33761       1    

In [109]:
# save to pickle
import pickle

with open('predictions_ca.pkl', 'wb') as f:
    pickle.dump(predictions_transformed, f)